In [264]:
import pandas as pd

In [2]:
med_data = pd.read_csv('../../data/claims_med_interns_2016-2018_20190304.csv')
med_data.columns

D:\Users\vhsu\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Member ID Encrypted', 'Claim ID', 'Service Line', 'Member Gender',
       'Age', 'Zip (5-digit)', 'MSA', 'County', 'State', 'Employee Type',
       'Relation', 'Line Of Business', 'Claim Status', 'Admit Type & Desc',
       'Admit Source & Desc', 'Discharge Status Code', 'Primary ICD Rollup',
       'Primary ICD Diagnosis Code', 'Primary ICD Diagnosis Desc',
       '2nd ICD Rollup', '2nd ICD Diagnosis Code', '2nd ICD Diagnosis Desc',
       '3rd ICD Rollup', '3rd ICD Diagnosis Code', '3rd ICD Diagnosis Desc',
       '4th ICD Rollup', '4th ICD Diagnosis Code', '4th ICD Diagnosis Desc',
       '5th ICD Rollup', '5th ICD Diagnosis Code', '5th ICD Diagnosis Desc',
       '6th ICD Rollup', '6th ICD Diagnosis Code', '6th ICD Diagnosis Desc',
       '7th ICD Rollup', '7th ICD Diagnosis Code', '7th ICD Diagnosis Desc',
       '8th ICD Rollup', '8th ICD Diagnosis Code', '8th ICD Diagnosis Desc',
       '9th ICD Rollup', '9th ICD Diagnosis Code', '9th ICD Diagnosis Desc',
       '10th IC

In [175]:
med = med_data[0:100]

In [375]:
def reshape(group):
    df = pd.DataFrame()
    
    # Demographic info
#     df['gender'] = group.head(1)['Member Gender']
#     df['age'] = group.head(1)['Age']
#     df['postal_code'] = group.head(1)['Zip (5-digit)']
    df['county'] = group.head(1)['County']
    
    df['claims_hypertension'] = 0
    df['claims_cardiovascular'] = 0
    df['claims_diabetes'] = 0
    df['claims_obesity'] = 0
    df['claims_cancer'] = 0
    df['claims_musculoskeletal'] = 0
    df['claims_copd'] = 0
    df['claims_cholesterol'] = 0
    df['total_claims'] = 0
    
    icd_codes = group['Primary ICD Diagnosis Code'] + group['2nd ICD Diagnosis Code'] + group['3rd ICD Diagnosis Code'] + group['4th ICD Diagnosis Code'] + group['5th ICD Diagnosis Code'] + group['6th ICD Diagnosis Code'] + group['7th ICD Diagnosis Code'] + group['8th ICD Diagnosis Code'] + group['9th ICD Diagnosis Code'] + group['10th ICD Diagnosis Code']
    icd_rollups = group['Primary ICD Rollup'] + group['2nd ICD Rollup'] + group['3rd ICD Rollup'] + group['4th ICD Rollup'] + group['5th ICD Rollup'] + group['6th ICD Rollup'] + group['7th ICD Rollup'] + group['8th ICD Rollup'] + group['9th ICD Rollup'] + group['10th ICD Rollup']
    
    # Set binary for conditions: 1 if condition exists, 0 otherwise
    df['binary_hypertension'] = any('I10' in s for s in icd_codes
    df['binary_cardiovascular'] = any('heart' in s for s in icd_rollups)
    df['binary_diabetes'] = any('Diabetes' in s for s in icd_rollups)
    df['binary_obesity'] = any('E66' in s for s in icd_codes)
    df['binary_cancer'] = any('Cancer' in s for s in icd_rollups)
    df['binary_musculoskeletal'] = any('musculoskeletal' in s for s in icd_rollups)0
    df['binary_copd'] = any('Chronic obstructive pulmonary disease' in s for s in icd_rollups)
    df['binary_cholesterol'] = any('E78' in s for s in icd_codes)
    
#     group.groupby(['Claim ID']).apply(claim_reshape, df=df)
    group.groupby(['Claim ID']).apply(claim_reshape, df=df)

    # Total number of conditions and claims
    total_conditions = df['binary_hypertension'] + df['binary_cardiovascular'] + df['binary_diabetes'] + df['binary_obesity'] + df['binary_cancer'] + df['binary_musculoskeletal'] + df['binary_copd'] + df['binary_cholesterol']
    df['total_conditions'] = total_conditions
    df['total_claims'] = f['total_claims']
    
    # Flags for nonused (claims were not filed for our conditions) and comorbidity (has more than one condition out of the ones we are interested in)
    df['flag_nonused'] = df['total_claims'].apply(lambda x: 1 if x == 0 else 0)
    df['flag_comorbidity'] = total_conditions.apply(lambda x: 1 if x > 1 else 0)

    return df

In [379]:
def claim_reshape(claim_group, df): 
    # Lists of all codes and rollups
    icd_codes = claim_group['Primary ICD Diagnosis Code'] + claim_group['2nd ICD Diagnosis Code'] + claim_group['3rd ICD Diagnosis Code'] + claim_group['4th ICD Diagnosis Code'] + claim_group['5th ICD Diagnosis Code'] + claim_group['6th ICD Diagnosis Code'] + claim_group['7th ICD Diagnosis Code'] + claim_group['8th ICD Diagnosis Code'] + claim_group['9th ICD Diagnosis Code'] + claim_group['10th ICD Diagnosis Code']
    icd_rollups = claim_group['Primary ICD Rollup'] + claim_group['2nd ICD Rollup'] + claim_group['3rd ICD Rollup'] + claim_group['4th ICD Rollup'] + claim_group['5th ICD Rollup'] + claim_group['6th ICD Rollup'] + claim_group['7th ICD Rollup'] + claim_group['8th ICD Rollup'] + claim_group['9th ICD Rollup'] + claim_group['10th ICD Rollup']
    
    count_claim = False
    
    if any('I10' in s for s in icd_codes):
        df['claims_hypertension'] += 1
        count_claim = True
    if any('heart' in s for s in icd_rollups):
        df['claims_cardiovascular'] += 1
        count_claim = True
    if any('Diabetes' in s for s in icd_rollups):
        df['claims_diabetes'] += 1
        count_claim = True
    if any('E66' in s for s in icd_codes):
        df['claims_obesity'] += 1
        count_claim = True
    if any('Cancer' in s for s in icd_rollups):
        df['claims_cancer'] += 1
        count_claim = True
    if any('musculoskeletal' in s for s in icd_rollups):
        df['claims_musculoskeletal'] += 1
        count_claim = True
    if any('Chronic obstructive pulmonary disease' in s for s in icd_rollups):
        df['claims_copd'] += 1
        count_claim = True
    if any('E78' in s for s in icd_codes):
        df['claims_cholesterol'] += 1
        count_claim = True
     
    if count_claim:
        df['total_claims'] += 1
        print(df['total_claims'])
        print(claim_group.head(1)['Claim ID'])


In [380]:
# Reshape data by grouping by member and year
reshaped_med_data = med.groupby(['Member ID Encrypted', 'Incurred Year']).apply(reshape)

0    1
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    2
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    1
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    2
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    1
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    2
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    1
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object
0    2
Name: total_claims, dtype: int64
0    180952D2AB000
Name: Claim ID, dtype: object


TypeError: incompatible index of inserted column with frame index

In [381]:
reshaped_med_data

county  claims_hypertension  \
Member ID Encrypted Incurred Year                                     
710087KT90EY0DZUV   2018          0     PIERCE                    0   
71008ADZ2VBBDM14C   2016          13      KING                    0   
7100UNBYH5SYEX19X   2016          14  KITTITAS                    0   
                    2018          21  KITTITAS                    0   
71069K607Y1CQ9BV5   2016          22      KING                    0   
                    2017          27      KING                    2   
                    2018          88      KING                    2   

                                      claims_cardiovascular  claims_diabetes  \
Member ID Encrypted Incurred Year                                              
710087KT90EY0DZUV   2018          0                       0                0   
71008ADZ2VBBDM14C   2016          13                      0                0   
7100UNBYH5SYEX19X   2016          14                      0                0   
                    2018          21                      0                0   
71069K607Y1CQ9BV5   2016          22                      0                0   
                    2017          27                      5                5   
                    2018          88                      2                2   

                                      claims_obesity  claims_cancer  \
Member ID Encrypted Incurred Year                                     
710087KT90EY0DZUV   2018          0                2              0   
71008ADZ2VBBDM14C   2016          13               0              0   
7100UNBYH5SYEX19X   2016          14               0              0   
                    2018          21               0              0   
71069K607Y1CQ9BV5   2016          22               0              0   
                    2017          27               0              0   
                    2018          88               0              0   

                                      claims_musculoskeletal  claims_copd  \
Member ID Encrypted Incurred Year                                           
710087KT90EY0DZUV   2018          0                        0            2   
71008ADZ2VBBDM14C   2016          13                       0            0   
7100UNBYH5SYEX19X   2016          14                       0            0   
                    2018          21                       0            0   
71069K607Y1CQ9BV5   2016          22                       0            0   
                    2017          27                       0            2   
                    2018          88                       0            2   

                                      claims_cholesterol  total_claims  ...  \
Member ID Encrypted Incurred Year                                       ...   
710087KT90EY0DZUV   2018          0                    0             2  ...   
71008ADZ2VBBDM14C   2016          13                   0             0  ...   
7100UNBYH5SYEX19X   2016          14                   0             0  ...   
                    2018          21                   0             0  ...   
71069K607Y1CQ9BV5   2016          22                   0             0  ...   
                    2017          27                   0             5  ...   
                    2018          88                   2             2  ...   

                                      binary_cardiovascular  binary_diabetes  \
Member ID Encrypted Incurred Year                                              
710087KT90EY0DZUV   2018          0                       0                0   
71008ADZ2VBBDM14C   2016          13                      0                0   
7100UNBYH5SYEX19X   2016          14                      0                0   
                    2018          21                      0                0   
71069K607Y1CQ9BV5   2016          22                      0                0   
                    2017          27                      1              